In [30]:
from datetime import timedelta, timezone, datetime
import jwt
from cryptography.hazmat.primitives.serialization import load_pem_private_key
from cryptography.hazmat.primitives.serialization import Encoding
from cryptography.hazmat.primitives.serialization import PublicFormat
from cryptography.hazmat.backends import default_backend
import base64
from getpass import getpass
import hashlib
import requests
import json
import os
import pandas as pd
pd.options.display.max_colwidth = 1000

# account parameters
SNOWFLAKE_ACCOUNT = "<ACCOUNT_NAME>" # must be capitalized
SNOWFLAKE_USER = "<USERNAME>" # must be capitalized
SNOWFLAKE_URL = "https://org-acc.snowflakecomputing.com"
PRIVATE_KEY_PATH = "/path/to/your/rsa_key.p8"

# service parameters
CORTEX_SEARCH_DATABASE = "<DATABASE>"
CORTEX_SEARCH_SCHEMA = "<SCHEMA>"
CORTEX_SEARCH_SERVICE = "<SERVICE>"

# columns to query in the service
COLUMNS = [
    "COL1",
    "COL2",
]

In [31]:
def generate_JWT_token():
    """
    https://docs.snowflake.com/en/developer-guide/sql-api/authenticating#generating-a-jwt-in-python
    Generate a valid JWT token from snowflake account name, user name, private key and private key passphrase.
    """
    # Prompt for private key passphrase
    def get_private_key_passphrase():
        return getpass('Private Key Passphrase: ')

    # Generate encoded public key
    with open(PRIVATE_KEY_PATH, 'rb') as pem_in:
        pemlines = pem_in.read()
        try:
            private_key = load_pem_private_key(pemlines, None, default_backend())
        except TypeError:
            private_key = load_pem_private_key(pemlines, get_private_key_passphrase().encode(), default_backend())
    public_key_raw = private_key.public_key().public_bytes(Encoding.DER, PublicFormat.SubjectPublicKeyInfo)
    sha256hash = hashlib.sha256()
    sha256hash.update(public_key_raw)
    public_key_fp = 'SHA256:' + base64.b64encode(sha256hash.digest()).decode('utf-8')

    # Generate JWT payload
    qualified_username = SNOWFLAKE_ACCOUNT + "." + SNOWFLAKE_USER
    now = datetime.now(timezone.utc)
    lifetime = timedelta(minutes=60)
    payload = {
        "iss": qualified_username + '.' + public_key_fp,
        "sub": qualified_username,
        "iat": now,
        "exp": now + lifetime
    }
    return jwt.encode(payload, key=private_key, algorithm="RS256")
    
jwt_token = generate_JWT_token()

headers = {
    'X-Snowflake-Authorization-Token-Type': 'KEYPAIR_JWT',
    'Content-Type': 'application/json',
    'Accept': 'application/json',
    'Authorization': f'Bearer {jwt_token}',
}

In [34]:
def query_service(query):
    """
    Query the specified Snowflake service with the given query string.
    """
    url = f"{SNOWFLAKE_URL}/api/v2/databases/{CORTEX_SEARCH_DATABASE}/schemas/{CORTEX_SEARCH_SCHEMA}/cortex-search-services/{CORTEX_SEARCH_SERVICE}:query"
    data = {
        "query": query,
        "columns": COLUMNS,
        "filter": "",
        "limit": 10
    }
    
    jwt_token = generate_JWT_token()
    headers = {
        'X-Snowflake-Authorization-Token-Type': 'KEYPAIR_JWT',
        'Content-Type': 'application/json',
        'Accept': 'application/json',
        'Authorization': f'Bearer {jwt_token}',
    }
    
    try:
        response = requests.post(url, headers=headers, json=data)
        response.raise_for_status()
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err} - Status code: {response.status_code}")
    except Exception as err:
        print(f"An error occurred: {err}")
    else:
        return response.json()

In [37]:
querystr = "<QUERY>"

df = pd.DataFrame(query_service(querystr)["results"])
df[COLUMNS].head()

,TALK_ID,SPEAKER_1,TITLE,DESCRIPTION,URL
0,220,Joseph Lekuton,A parable for Kenya,"Joseph Lekuton, a member of parliament in Kenya, starts with the story of his remarkable education, then offers a parable of how Africa can grow. His message of hope has never been more relevant.",https://www.ted.com/talks/joseph_lekuton_a_parable_for_kenya/
1,2221,Boniface Mwangi,The day I stood up alone,"Photographer Boniface Mwangi wanted to protest against corruption in his home country of Kenya. So he made a plan: He and some friends would stand up and heckle during a public mass meeting. But when the moment came ... he stood alone. What happened next, he says, showed him who he truly was. As he says, ""There are two most powerful days in your life. The day you are born, and the day you discover why."" Graphic images.",https://www.ted.com/talks/boniface_mwangi_the_day_i_stood_up_alone/
2,523,Erik Hersman,Reporting crisis via texting,"At TEDU 2009, Erik Hersman presents the remarkable story of Ushahidi, a GoogleMap mashup that allowed Kenyans to report and track violence via cell phone texts following the 2008 elections, and has evolved to continue saving lives in other countries.",https://www.ted.com/talks/erik_hersman_reporting_crisis_via_texting/
3,2653,Charity Wayua,A few ways to fix a government,"Charity Wayua put her skills as a cancer researcher to use on an unlikely patient: the government of her native Kenya. She shares how she helped her government drastically improve its process for opening up new businesses, a crucial part of economic health and growth, leading to new investments and a World Bank recognition as a top reformer.",https://www.ted.com/talks/charity_wayua_a_few_ways_to_fix_a_government/
4,21033,Mary Maker,Why I fight for the education of refugee girls (like me),"After fleeing war-torn South Sudan as a child, Mary Maker found security and hope in the school at Kenya's Kakuma Refugee Camp. Now a teacher of young refugees herself, she sees education as an essential tool for rebuilding lives -- and empowering a generation of girls who are too often denied entrance into the classroom. ""For the child of war, an education can turn their tears of loss into a passion for peace,"" Maker says.",https://www.ted.com/talks/mary_maker_why_i_fight_for_the_education_of_refugee_girls_like_me/
